# 斷詞

## 英文斷詞

In [ ]:
# 英文斷詞
from tensorflow.keras.preprocessing.text import text_to_word_sequence
print("英文斷詞：", text_to_word_sequence("I love jogging, and you?"))

英文斷詞： ['i', 'love', 'jogging', 'and', 'you']


## 中文斷詞

In [ ]:
# Install jieba（結巴）
!pip install jieba

# Get the Tokenization Dictionary for Traditional Chinese
import os
Dictionary_File = 'dict.txt.big'

if not os.path.isfile(Dictionary_File):
    os.system('wget https://raw.githubusercontent.com/cnchi/datasets/master/' + Dictionary_File)

# Get the Stop Words File for Traditional Chinese
StopWords_File = "stopWords_big5.txt"

if not os.path.isfile(StopWords_File):
    os.system('wget https://raw.githubusercontent.com/cnchi/datasets/master/' + StopWords_File)

In [ ]:
import jieba

# Set Dictionary for Traditional Chinese
# jieba.set_dictionary(Dictionary_File)

# Tokenization
result = list(jieba.cut("我喜歡跑步，你呢？"))
print("中文斷詞（有標點）：", result)

# Remove Stop Words from Set
stopWords = set("$!&#%\()+-*/_,. 　?:;'\"<=>^`|~[]{}’0123456789?_“”、。《》！，：；？「」（）")
print("中文斷詞（無標點）：", [word for word in result if word not in stopWords])

# Remove Stop Words from Files
stopWords = set()
with open(StopWords_File, "rt", encoding="utf-8") as f:
  for line in f:
    line = line.strip() # Remove trailing \n
    stopWords.add(line)
print("中文斷詞（更精簡）：", [word for word in result if word not in stopWords])

中文斷詞（有標點）： ['我', '喜歡', '跑步', '，', '你', '呢', '？']
中文斷詞（無標點）： ['我', '喜歡', '跑步', '你', '呢']
中文斷詞（更精簡）： ['喜歡', '跑步']


# 文字數位化

In [ ]:
# Create a Tokenizer object
from tensorflow.keras.preprocessing.text import Tokenizer

tk = Tokenizer(
        num_words=None,
        filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~\t\n',
        lower=True,
        split=' ',
        char_level=False,
        oov_token='NiD'
    )

In [ ]:
# Create Mapping by Corpus
corpus = ["I love jogging, and you?",
      "I love reading!"]
tk.fit_on_texts(corpus)

# Show the Mapping Table
print(tk.word_index)    # WORD vs. NUMBER
print(tk.index_word)    # NUMBER vs. WORD

{'NiD': 1, 'i': 2, 'love': 3, 'jogging': 4, 'and': 5, 'you': 6, 'reading': 7}
{1: 'NiD', 2: 'i', 3: 'love', 4: 'jogging', 5: 'and', 6: 'you', 7: 'reading'}


In [ ]:
# Test for Mapping Text into Sequence
input_text = ["I love jogging!",
        "and I love reading, too!"]

seq = tk.texts_to_sequences(input_text)
print(seq)

# Test for Mapping Sequence into Text
text = tk.sequences_to_texts(seq)
print(text)

[[2, 3, 4], [5, 2, 3, 7, 1]]
['i love jogging', 'and i love reading NiD']


# 序列對齊（Sequence Alignment）

In [ ]:
# Create a Sequence Padding Object
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_seq = pad_sequences(
        sequences=seq,      #要被對齊的序列
        maxlen=5,         #一律調整成五個字
        dtype="int32",       #使用整數作為輸出型態
        padding="pre",      #補字的話補面
        truncating="post",    #要砍字砍後面
        value=0         #不足一律補0
    )

print(padded_seq)

[[0 0 2 3 4]
 [5 2 3 7 1]]


# 編碼（Encoding）

In [ ]:
# One-Hot Encoding
from tensorflow.keras.utils import to_categorical

print("獨熱編碼 -------------")
print(to_categorical(padded_seq))

獨熱編碼 -------------
[[[1. 0. 0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [ ]:
# Multi-Hot Encoding
print("多熱編碼 -------------")
print(tk.texts_to_matrix(input_text))

多熱編碼 -------------
[[0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 1. 1. 1. 0. 1. 0. 1.]]


In [ ]:
# Word Embedding
import tensorflow as tf
from tensorflow.keras import layers

emb = layers.Embedding(8, 3)

# tf.constant(): Convert immediate values into tensor
result = emb(tf.constant(padded_seq))
print("詞向量嵌入 -------------")
print(result.numpy())

詞向量嵌入 -------------
[[[ 0.04907269  0.03253496  0.02634681]
  [ 0.04907269  0.03253496  0.02634681]
  [-0.01797484  0.04377304 -0.02397053]
  [-0.0417596  -0.00967396  0.03428981]
  [-0.04945678  0.03748863 -0.03949701]]

 [[ 0.04733828  0.00326739 -0.02105859]
  [-0.01797484  0.04377304 -0.02397053]
  [-0.0417596  -0.00967396  0.03428981]
  [-0.00888322 -0.04275189 -0.03454363]
  [ 0.04514853 -0.03064466  0.00182449]]]
